In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.1 MB/s eta 0:00:00


In [5]:
import docx
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import random

def add_checlist_to_docx(document, checklist_item):
    from docx.oxml.shared import OxmlElement, qn

    paragraph = document.add_paragraph()
    tag = paragraph._p

    sdt = OxmlElement('w:sdt')
    sdtPr = OxmlElement('w:sdtPr')
    checkbox = OxmlElement('w14:checkbox')
    checked = OxmlElement('w14:checked')
    checked.set(qn('w14:val'), '0')
    checkedState = OxmlElement('w14:checkedState')
    checkedState.set(qn('w14:val'), '2612')
    checkedState.set(qn('w14:font'), 'MS Gothic')
    uncheckedState = OxmlElement('w14:uncheckedState')
    uncheckedState.set(qn('w14:val'), '2610')
    uncheckedState.set(qn('w14:font'), 'MS Gothic')

    sdtContent = OxmlElement('w:sdtContent')
    r_box = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')
    rFonts = OxmlElement('w:rFonts')
    rFonts.set(qn('w:ascii'), 'MS Gothic')
    rFonts.set(qn('w:eastAsia'), 'MS Gothic')
    rFonts.set(qn('w:hAnsi'), 'MS Gothic')
    rFonts.set(qn('w:hint'), 'eastAsia')
    t_box = OxmlElement('w:t')

    r_text = OxmlElement('w:r')
    t_text = OxmlElement('w:t')
    t_text.set(qn('xml:space'), 'preserve')

    checkbox.append(checked)
    checkbox.append(checkedState)
    checkbox.append(uncheckedState)
    sdtPr.append(checkbox)
    sdt.append(sdtPr)
    rPr.append(rFonts)
    t_box.text = '☐'
    r_box.append(rPr)
    r_box.append(t_box)
    sdtContent.append(r_box)
    sdt.append(sdtContent)
    tag.append(sdt)
    t_text.text = checklist_item
    r_text.append(t_text)
    tag.append(r_text)
    return

# create an instance of a word document
doc = docx.Document()
is_over = False
with open('/content/drive/My Drive/Ansible Health/MarkdownMeetingNotes.md', 'r') as file:
      # format the doc line by line
      for line in file:
        if(line.startswith("# ")):
            doc.add_heading(line[2:], 1)
        elif(line.startswith("## ")):
            doc.add_heading(line[3:], 2)
        elif(line.startswith("### ")):
            doc.add_heading(line[4:], 3)
        elif(line.startswith("* ")):
            doc.add_paragraph(line[2:], style='List Number')
        elif(line.startswith("  * ")):
            doc.add_paragraph(line[4:], style='List Number 2')
        elif(line.startswith("    * ")):
            doc.add_paragraph(line[6:], style='List Number 3')
        elif(line.startswith("- [ ] ")):
          # special format for @ is a newline for the list item
            name = line.split("@",1)[1]
            name = name.split(":")[0]
            task = line.split(":",1)[1]
            add_checlist_to_docx(doc, "@"+name+":")
            doc.add_paragraph("     "+task)
        elif(line.startswith("---") or is_over):
          # special format for footer information is italics
          is_over = True
          p = doc.add_paragraph()
          p.add_run(line).italic = True;
        else:
          doc.add_paragraph(line)

# now save the document to a location
doc.save('/content/drive/My Drive/Ansible Health/MeetingNotes.docx')